# 第七章 Gradio 的介绍与前端界面的搭建 💬

 - [一、Gradio 简介](#一、Gradio-简介)
     - [1.1 简单示例](#1.1-简单示例)
 - [二、使用 `gr.Chatbot()` 来助力聊天!](#二、使用-`gr.Chatbot()`-来助力聊天!)
 - [三、 llm 通过本地数据库进行回答](#三、-llm-通过本地数据库进行回答)
     - [3.1 封装函数](#3.1-封装函数)
     - [3.2 组装 gradio 界面](#3.2-组装-gradio-界面)


Gradio 是一种快速便捷的方法，可以直接在 **Python 中通过友好的 Web 界面演示机器学习模型**。在本课程中，我们将学习*如何使用它为生成式人工智能应用程序构建用户界面*。在构建了应用程序的机器学习或生成式人工智能后，如果你想构建一个demo给其他人看，也许是为了获得反馈并推动系统的改进，或者只是因为你觉得这个系统很酷，所以想演示一下：Gradio 可以让您通过 Python 接口程序快速实现这一目标，而无需编写任何前端、网页或 JavaScript 代码。
加载 HF API 密钥和相关 Python 库

首先我们导入使用到的库

In [2]:
# 导入必要的库
import os                # 用于操作系统相关的操作，例如读取环境变量
import io                # 用于处理流式数据（例如文件流）
import IPython.display   # 用于在IPython环境中显示数据，例如图片
import requests          # 用于进行HTTP请求，例如GET和POST请求
import zhipuai
from zhipuai_llm import ZhipuAILLM

# 设置请求的默认超时时间为60秒
requests.adapters.DEFAULT_TIMEOUT = 60

# 导入dotenv库的函数
# dotenv允许您从.env文件中读取环境变量
# 这在开发时特别有用，可以避免将敏感信息（如API密钥）硬编码到代码中
from dotenv import load_dotenv, find_dotenv

# 寻找.env文件并加载它的内容
# 这允许您使用os.environ来读取在.env文件中设置的环境变量
_ = load_dotenv(find_dotenv())

# 从环境变量中读取'ZHIPUAI_API_KEY'并将其存储在hf_api_key变量中
zhipuai.api_key = os.environ['ZHIPUAI_API_KEY']

In [3]:
# 助手函数
llm = ZhipuAILLM(model="chatglm_std", temperature=0)

## 一、Gradio 简介

Gradio 可以包装几乎任何 Python 函数为易于使用的用户界面。

常用的基础模块构成如下：

- 应用界面：gr.Interface(简易场景), gr.Blocks(定制化场景)

- 输入输出：gr.Image(图像), gr.Textbox(文本框), gr.DataFrame(数据框), gr.Dropdown(下拉选项), gr.Number(数字), gr.Markdown(Markdown), gr.Files(文件)

- 控制组件：gr.Button(按钮)

- 布局组件：gr.Tab(标签页), gr.Row(行布局), gr.Column(列布局)

大部分功能模块都可以通过以下三个参数进行初始化：

- fn：包装的函数
- inputs：输入组件类型，（例如：“text”、"image）
- ouputs：输出组件类型，（例如：“text”、"image）

### 1.1 简单示例

- fn=generate: 这是用于处理输入的函数，即文本生成函数 generate。
- inputs=[
        gr.Textbox(label="Prompt"),
        gr.Slider(label="Temperature", value=0,  maximum=1, minimum=0)。
    ]: 这定义了模型的输入。
    使用 gr.Textbox 部件来以文本框的形式显示输入的内容描述，label 参数设置了输入部件的标签为 prompt。
    使用 gr.Slider 部件以滑动条的形式来显示输入的内容描述，label 参数设置了输入部件的标签为 temperature。
- outputs=[gr.Textbox(label="Caption")]: 这定义了输出部分。使用 gr.Textbox 部件来显示生成的内容描述，label 参数设置了输出部件的标签。
- title="Chat Robot": 这是界面的标题，将显示在界面的顶部。
- description="Local Knowledge Base Q&A with llm ": 这是界面的描述，提供有关界面功能的更多信息。
- allow_flagging="never": 这设置了不允许标记内容，确保不会显示标记不恰当内容的选项。

In [4]:
# 导入所需的库
import gradio as gr  # 用于创建Web界面
import os  # 用于与操作系统交互，如读取环境变量

# 定义一个函数来根据输入生成文本
def generate(input, slider):
    # 使用预定义的client对象的generate方法，从输入生成文本
    # slider的值限制生成的token的数量
    output = llm.predict(input, temperature=slider)
    return output  # 返回生成的文本

# 创建一个Web界面
# 输入：一个文本框和一个滑块
# 输出：一个文本框显示生成的文本
demo = gr.Interface(
    fn=generate, 
    inputs=[
        gr.Textbox(label="Prompt"),  # 文本输入框
        gr.Slider(label="Temperature", value=0,  maximum=1, minimum=0)  # 滑块用于选择模型的 temperature
    ], 
    outputs=[gr.Textbox(label="Completion")],  # 显示生成文本的文本框
    title="Chat Robot",  # 界面标题
    description="Local Knowledge Base Q&A with llm",  # 界面描述
    # allow_flagging="never", 
)

# 关闭可能已经启动的任何先前的gradio实例
gr.close_all()

# 启动Web界面
# 使用环境变量PORT1作为服务器的端口号
# demo.launch(share=True, server_port=int(os.environ['PORT1']))
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


现在我们已经搭建了一个非常简单的 Gradio 界面，它有一个文本框输入和一个输出。我们已经可以非常简单地向 LLM 提问。但我们还是不能对话，因为如果你再问一个后续问题，它就无法理解或保留上下文。

因此，基本上我们要做的是，向模型发送我们之前的问题、它自己的回答以及后续问题。但建立所有这些都有点麻烦。这就是 Gradio 聊天机器人组件的作用所在，因为它允许我们简化向模型发送对话历史记录的过程。

因此，我们要解决这个问题。为此，我们将引入一个新的 Gradio 组件--Gradio Chatbot。

## 二、使用 `gr.Chatbot()` 来助力聊天!

让我们开始使用 Gradio Chatbot 组件。这里实例化了一个带有文本框提示和提交按钮的 Gradle ChatBot 组件，是一个非常简单的用户界面。但我们现在还不是在和 LLM 聊天。

我们必须格式化聊天prompt。此处正在定义这个格式化聊天prompt函数。
在这里，我们要做的就是使其包含聊天历史记录，这样 LLM 就能知道上下文。
但这还不够。我们还需要告诉它，哪些信息来自用户，哪些信息来自 LLM 本身，也就是我们正在调用的助手。
因此，我们设置了格式聊天prompt功能，在聊天记录的每一轮中，都包含一条用户信息和一条助手信息，以便我们的模型能准确回答后续问题。
现在，我们要将格式化的prompt传递给我们的 API。

相比 Interface，Blocks 提供了一个低级别的 API，用于设计具有更灵活布局和数据流的网络应用。Blocks 允许控制组件在页面上出现的位置，处理复杂的数据流（例如，输出可以作为其他函数的输入），并根据用户交互更新组件的属性可见性。可以定制更多组件。

In [5]:
# 定义一个函数，用于格式化聊天提示。
def format_chat_prompt(message, chat_history):
    # 初始化一个空字符串，用于存放格式化后的聊天提示。
    prompt = ""
    # 遍历聊天历史记录。
    for turn in chat_history:
        # 从聊天记录中提取用户和机器人的消息。
        user_message, bot_message = turn
        # 更新提示，加入用户和机器人的消息。
        prompt = f"{prompt}\nUser: {user_message}\nAssistant: {bot_message}"
    # 将当前的用户消息也加入到提示中，并预留一个位置给机器人的回复。
    prompt = f"{prompt}\nUser: {message}\nAssistant:"
    # 返回格式化后的提示。
    return prompt

# 定义一个函数，用于生成机器人的回复。
def respond(message, chat_history):
    # 调用上面的函数，将用户的消息和聊天历史记录格式化为一个提示。
    formatted_prompt = format_chat_prompt(message, chat_history)
    # 使用llm对象的predict方法生成机器人的回复（注意：llm对象在此代码中并未定义）。
    bot_message = llm.predict(formatted_prompt,
                                  max_new_tokens=1024,
                                  stop_sequences=["\nUser:", ""])
    # 将用户的消息和机器人的回复加入到聊天历史记录中。
    chat_history.append((message, bot_message))
    # 返回一个空字符串和更新后的聊天历史记录（这里的空字符串可以替换为真正的机器人回复，如果需要显示在界面上）。
    return "", chat_history

# 下面的代码是设置Gradio界面的部分。

# 使用Gradio的Blocks功能定义一个代码块。
with gr.Blocks() as demo:
    # 创建一个Gradio聊天机器人组件，设置其高度为240。
    chatbot = gr.Chatbot(height=240) 
    # 创建一个文本框组件，用于输入提示。
    msg = gr.Textbox(label="Prompt")
    # 创建一个提交按钮。
    btn = gr.Button("Submit")
    # 创建一个清除按钮，用于清除文本框和聊天机器人组件的内容。
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    # 设置按钮的点击事件。当点击时，调用上面定义的respond函数，并传入用户的消息和聊天历史记录，然后更新文本框和聊天机器人组件。
    btn.click(respond, inputs=[msg, chatbot], outputs=[msg, chatbot])
    # 设置文本框的提交事件（即按下Enter键时）。功能与上面的按钮点击事件相同。
    msg.submit(respond, inputs=[msg, chatbot], outputs=[msg, chatbot]) 

# 关闭所有已经存在的Gradio实例。
gr.close_all()
# 启动新的Gradio应用，设置分享功能为True，并使用环境变量PORT3指定服务器端口。
# demo.launch(share=True, server_port=int(os.environ['PORT3']))
demo.launch()

Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


现在，我们的问答机器人可以回答后续问题了。
我们可以看到，我们向它发送了上下文。我们向它发送了信息，然后要求它完成。一旦我们进入另一个迭代循环，我们就会向它发送我们的整个上下文，然后要求它完成。这很酷。但是，如果我们一直这样迭代下去，那么模型在一次对话中所能接受的信息量就会达到极限，因为我们总是给它越来越多的之前对话的内容。

这里，我们创建了一个简单但功能强大的用户界面，用于与LLM聊天。如果需要进一步Gradio 所能提供的最佳功能，我们可以创建一个包含更多功能的用户界面。

## 三、 llm 通过本地数据库进行回答

现在我们可以将本地数据库的内容接入进来，让 llm 通过本地数据库进行回答。

In [7]:
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from zhipuai_embedding import ZhipuAIEmbeddings

from langchain.llms import OpenAI
from langchain.llms import HuggingFacePipeline
from zhipuai_llm import ZhipuAILLM

In [8]:
from langchain.prompts import PromptTemplate

In [38]:
persist_directory = 'docs/chroma/knowledge_base'

In [13]:
# 导入检索式问答链
from langchain.chains import RetrievalQA
def chat_with_db(query, chat_history):

    llm = ZhipuAILLM(model="chatglm_std")

    embedding = ZhipuAIEmbeddings()
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
    # Build prompt
    template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
    {context}
    问题：{question}
    有用的回答："""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
    # qa_chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever())
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(),
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
    )
    result = qa_chain({"query": query, "context": chat_history})
    
    # 将用户的消息和机器人的回复加入到聊天历史记录中。
    chat_history.append((query, result['result']))
    # 返回一个空字符串和更新后的聊天历史记录（这里的空字符串可以替换为真正的机器人回复，如果需要显示在界面上）。
    return "", chat_history

In [15]:
# 使用Gradio的Blocks功能定义一个代码块。
with gr.Blocks() as demo:
    # 创建一个Gradio聊天机器人组件，设置其高度为240。
    chatbot = gr.Chatbot(height=240) 
    # 创建一个文本框组件，用于输入提示。
    msg = gr.Textbox(label="Prompt")
    # 创建一个提交按钮。
    btn = gr.Button("Submit")
    # 创建一个清除按钮，用于清除文本框和聊天机器人组件的内容。
    clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")

    # 设置按钮的点击事件。当点击时，调用上面定义的 chat_with_db 函数，并传入用户的消息和聊天历史记录，然后更新文本框和聊天机器人组件。
    btn.click(chat_with_db, inputs=[msg, chatbot], outputs=[msg, chatbot])
    # 设置文本框的提交事件（即按下Enter键时）。功能与上面的按钮点击事件相同。
    msg.submit(chat_with_db, inputs=[msg, chatbot], outputs=[msg, chatbot]) 

# 关闭所有已经存在的Gradio实例。
gr.close_all()
# 启动新的Gradio应用，设置分享功能为True，并使用环境变量PORT3指定服务器端口。
# demo.launch(share=True, server_port=int(os.environ['PORT3']))
demo.launch()

Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


### 3.1 封装函数

现在我们可以将所需的功能函数进行封装，将其与 gradio 进行绑定。

1. 上传文档
2. 初始化向量数据库
3. 向 LLM 提问
4. 清空对话

In [69]:
from langchain.document_loaders import PyMuPDFLoader

In [153]:
def init_knowledge_db(file):
    """
    该函数用于加载 PDF 文件，切分文档，生成文档的嵌入向量，创建向量数据库。

    参数:
    file (str): 要加载的 PDF 文件路径。

    返回:
    vectordb (vectordb): 创建的数据库。
    """
    # 载入文档
    loader = PyMuPDFLoader(file.name)
    documents = loader.load()
    # 切分文档
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=150)
    split_docs = text_splitter.split_documents(documents)
    # 定义 Embeddings
    embeddings = ZhipuAIEmbeddings()
    # 根据数据创建向量数据库
    vectordb = Chroma.from_documents(
        documents=split_docs[:3], # 为了速度，只选择了前 100 个切分的 doc 进行生成。
        embedding=embedding,
        persist_directory=persist_directory  # 允许我们将persist_directory目录保存到磁盘上
    )
    return vectordb 


In [154]:
def load_knowledge_db(path, embeddings):
    vectordb = Chroma.from_documents(
        embeddings=embeddings,
        persist_directory=path.name  
    )
    vectordb.persist()

In [155]:
def presit_knowledge_db(vectordb):
    vectordb.persist()

In [180]:
def chat_with_db(query, chat_history, history_len, top_k, temperature):

    llm = ZhipuAILLM(model="chatglm_std", temperature=temperature)
    embedding = ZhipuAIEmbeddings()
    vectordb = Chroma(
        persist_directory=persist_directory,
        embedding_function=embedding
    )
    # Build prompt
    template = """使用以下上下文片段来回答最后的问题。如果你不知道答案，只需说不知道，不要试图编造答案。答案最多使用三个句子。尽量简明扼要地回答。在回答的最后一定要说"感谢您的提问！"
    {context}
    问题：{question}
    有用的回答："""
    QA_CHAIN_PROMPT = PromptTemplate.from_template(template)
    # qa_chain = ConversationalRetrievalChain.from_llm(llm, vectordb.as_retriever())
    qa_chain = RetrievalQA.from_chain_type(
        llm,
        retriever=vectordb.as_retriever(
                search_kwargs={"k": top_k}),
        return_source_documents=True,
        chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    )
    # 限制 history 的记忆长度
    chat_history = chat_history[-history_len:] if history_len > 0 else []

    result = qa_chain({"query": query, "context": chat_history})
    
    # 将用户的消息和机器人的回复加入到聊天历史记录中。
    chat_history.append((query, result['result']))
    # 返回一个空字符串和更新后的聊天历史记录（这里的空字符串可以替换为真正的机器人回复，如果需要显示在界面上）。
    return "", chat_history

### 3.2 组装 gradio 界面

我们可以在 gradio.Markdown 中增加一些描述性写法。

我们可以配置些高级参数，
- 向量检索的数量（top_k）：从向量数据库返回的最相关文档的数量，LLM 会根据返回的文档生成回答。
- 聊天历史的长度（history_len）：使聊天历史保持一定的长度，防止过大消耗过多的 token。
- 温度（temperature）：温度基本上就是你希望模型的变化程度。因此，如果将温度设为零，模型就会倾向于始终对相同的输入做出相同的反应。所以同样的问题，同样的答案。温度越高，信息的变化就越多。但如果温度过高，它就会开始给出无意义的答案。

In [186]:
block = gr.Blocks()
with block as demo:
    gr.Markdown("""<h1><center>Chat Robot</center></h1>
    <center>Local Knowledge Base Q&A with llm</center>
    """)
    with gr.Row():
        with gr.Column(scale=4):
            chatbot = gr.Chatbot(height=240) 
            # 创建一个文本框组件，用于输入提示。
            msg = gr.Textbox(label="Prompt/问题")

            # 设置按钮的点击事件。当点击时，调用上面定义的 chat_with_db 函数，并传入用户的消息和聊天历史记录，然后更新文本框和聊天机器人组件。
            btn.click(chat_with_db, inputs=[msg, chatbot, history_len, top_k, temperature], outputs=[msg, chatbot])
            # 设置文本框的提交事件（即按下Enter键时）。功能与上面的按钮点击事件相同。
            msg.submit(chat_with_db, inputs=[msg, chatbot, history_len, top_k, temperature], outputs=[msg, chatbot]) 

            with gr.Row():
                # 创建一个提交按钮。
                btn = gr.Button("Submit")
                # 创建一个清除按钮，用于清除文本框和聊天机器人组件的内容。
                clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")
    
        with gr.Column(scale=1):
            model_argument = gr.Accordion("问答参数配置", )
            with model_argument:
                top_k = gr.Slider(1,
                                10,
                                value=3,
                                step=1,
                                label="vector db search top k",
                                interactive=True)

                history_len = gr.Slider(0,
                                        5,
                                        value=3,
                                        step=1,
                                        label="history len",
                                        interactive=True)

                temperature = gr.Slider(0,
                                        1,
                                        value=0.01,
                                        step=0.01,
                                        label="llm temperature",
                                        interactive=True)

            file = gr.File(label='请上传知识库文件',
                            file_types=['.txt', '.md', '.docx', '.pdf'])
            with gr.Row():
                # # 创建一个提交按钮。
                # btn = gr.Button("Submit")
                # # 创建一个清除按钮，用于清除文本框和聊天机器人组件的内容。
                # clear = gr.ClearButton(components=[msg, chatbot], value="Clear console")
                init_db = gr.Button("知识库文件向量化")
                
        init_db.click(
            init_knowledge_db,
            show_progress=True,
            inputs=[file],
            outputs=[],
        )

        # 设置按钮的点击事件。当点击时，调用上面定义的 chat_with_db 函数，并传入用户的消息和聊天历史记录，然后更新文本框和聊天机器人组件。
        btn.click(chat_with_db, inputs=[msg, chatbot, history_len, top_k, temperature], outputs=[msg, chatbot])
        # 设置文本框的提交事件（即按下Enter键时）。功能与上面的按钮点击事件相同。
        msg.submit(chat_with_db, inputs=[msg, chatbot, history_len, top_k, temperature], outputs=[msg, chatbot]) 

    gr.Markdown("""提醒：<br>
    1. 使用时请先上传自己的知识文件，并且文件中不含某些特殊字符，否则将返回error. <br>
    """)
# threads to consume the request
gr.close_all()
# 启动新的Gradio应用，设置分享功能为True，并使用环境变量PORT3指定服务器端口。
# demo.launch(share=True, server_port=int(os.environ['PORT3']))
demo.launch()

Closing server running on port: 7860
Running on local URL:  http://127.0.0.1:7911

To create a public link, set `share=True` in `launch()`.


现在我们已经实现了 《llm 通过本地数据库进行回答》的基本功能和界面。快去进行自己的尝试吧。

后续还可以增加很多功能优化：
  1. 增加模型选择功能
  2. embedding 选择功能
  3. 更细致化的模型配置